In [1]:
import cv2
import re
import json
from glob import glob
from pytesseract import pytesseract

In [2]:
arqs = sorted(glob(r'patents/*png'))

In [3]:
class Arquive:
    def __init__(self, arq):
        self.arq = arq
        self.text = pytesseract.image_to_string(self.arq)
        self.size = len(str(self.text))
        self.img = cv2.imread(arq)
        self.shape = self.img.shape
        self.cut_text = pytesseract.image_to_string(
            self.img[int(self.shape[0] / 10):int(self.shape[0] /6), :int(self.shape[1] / 2)])

    def get_patent(self):
        patent_number = re.compile(r'(?<=Patent N).*')
        check = patent_number.findall(self.text)
        num = re.compile(r'\d')
        check_num = num.findall(str(check))

        return int(''.join(check_num))

    def get_dates(self):
        patent_date = re.compile(
            r'[t|d]:(.?){4}([A-Z]+[a-z]{2}.).([0-9]+.).([0-9]{4})')
        check_dates = patent_date.findall(self.text)

        patent = (''.join(list(check_dates[1][1:])).replace(',', '-').replace('.', '-').replace('Jan', '1').replace('Feb', '2').replace('Mar', '3').replace('Apr', '4').replace('May', '5').replace('Jun','6').replace('Jul', '7').replace('Aug', '8').replace('Sep', '9').replace('Oct','10').replace('Nov', '11').replace( "Dec", '12').replace(' ', '-'))
        filed = (''.join(list(check_dates[0][1:])).replace(',', '-').replace('.', '-').replace('Jan', '1').replace('Feb', '2').replace('Mar', '3').replace('Apr', '4').replace('May', '5').replace('Jun','6').replace('Jul', '7').replace('Aug', '8').replace('Sep', '9').replace('Oct','10').replace('Nov', '11').replace( "Dec", '12').replace(' ', '-'))
        return patent, filed

    def get_title(self):
        patent_title = re.compile(
            r'(?<![, ])[ A-Z|\-A-Z]?[A-Z]{3,}(?![\.|a-z|\]|0-9|])')
        check = patent_title.findall(self.cut_text)
        return (' '.join(check)).title()

    def get_inventor(self):
        patent_inventor = re.compile(r'(?<=Inventor:).*')
        check = patent_inventor.findall(self.text)
        str(check).split(',')
        name = check[0].split(',')

        return name[0].strip()

In [4]:
def get_info(arq):
    dic = dict()
    arqx = Arquive(arq)
    filedD, patentD = arqx.get_dates()
    dic["arq"]=arqx.arq
    dic["patent_number"]=arqx.get_patent()
    dic["date_of_patent"]=patentD 
    dic["patent_title"]=arqx.get_title() 
    dic["inventor_name"]=arqx.get_inventor()
    dic["filed_data"]=filedD
    
    return dic

In [5]:
with open('patents.json', 'w'):#limpa o arquivo
    pass
    
dic_arr = list()
for arq in arqs:
    dic_arr.append(get_info(arq))
    
with open("patents.json", "a+") as outfile:#escreve no arquivo
    json.dump(dic_arr, outfile)